# Title: Terrestrial Ecoregions of the World (2001) WWF

### Description
This map depicts the 825 terrestrial ecoregions of the globe. Ecoregions are relatively large units of land containing distinct assemblages of natural communities and species, with boundaries that approximate the original extent of natural communities prior to major land-use change. This comprehensive, global map provides a useful framework for conducting biogeographical or macroecological research, for identifying areas of outstanding biodiversity and conservation priority, for assessing the representation and gaps in conservation efforts worldwide, and for communicating the global distribution of natural communities on earth. We have based ecoregion delineations on hundreds of previous biogeographical studies, and refined and synthesized existing information in regional workshops over 10 years to assemble the global dataset. Ecoregions are nested within two higher-order classifications: biomes (14) and biogeographic realms (8). Together, these nested classification levels provide a framework for comparison among units and the identification of representative habitats and species assemblages. Ecoregions have increasingly been adopted by research scientists, conservation organizations, and donors as a framework for analyzing biodiversity patterns, assessing conservation priorities, and directing effort and support (Ricketts et al. 1999a; Wikramanayake et al. 2001; Ricketts et al. 1999b; Olson & Dinerstein 1998; Groves et al. 2000; Rosenzweig et al. 2003; and Luck et al. 2003). More on the approach to ecoregion mapping, the logic and design of the framework, and previous and potential uses are discusses in Olson et al. (2001) and Ricketts et al. (1999a).

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/<br>

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Year: </b>2001<br>
<b>Size:</b>  157 MB

### Original source
Original Source: http://databasin.org/datasets/68635d7c77f1475f9b6c1d1dbe0a4c4c Accessed 13/12/2020 (login required, free access)

### Licence
This work is licensed under a Creative Commons Attribution-NonCommercial 3.0 License.<br> Please read the 'Terms of Use' text file from original source.

### Citation
Olson, D.M., E. Dinerstein, E.D. Wikramanayake, N.D. Burgess, G.V.N. Powell, E.C. Underwood, J.A. D'Amico, I. Itoua, H.E. Strand, J.C. Morrison, C.J. Loucks, T.F. Allnutt, T.H. Ricketts, Y. Kura, J.F. Lamoreux, W.W. Wettengel, P. Hedao, and K.R. Kassem. Terrestrial Ecoregions of the World: A New Map of Life on Earth (PDF, 1.1M) BioScience 51:933-938.

### Original format
Global coverage, vector, shapefile<br>
Cordinate system EPSG: 4326 (WGS84)

### Metadata
Display Attribute: ECO_SYM <br>
Primary Attribute: ECO_NAME <br>
Other metadata: https://databasin.org/datasets/68635d7c77f1475f9b6c1d1dbe0a4c4c

### Notes
Original data source is in WGS_1984_Web_Mercator_Auxiliary_Sphere (EPSG:3857). It appears to be 100-1000m offset in some places so please consider the boundaries of the data to be only be +/-1000m location accuracy. Some topological error requires fixing see code below.

### Processing
Repair geometry (non simple), fix topologial error (remove overlaps, fill slither gaps), convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
import arcpy
import os

#folder of feature classes to process
in_folder = r"C:\data\terrestrial_ecosystems"
scr_folder = os.path.join(in_folder, "scratch.gdb")
workspace = in_folder
out_folder = r"C:\data\json"
outjson = os.path.join(out_folder, fcname)

# Set environments
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True

# Execute CreateFileGDB
scr = arcpy.CreateFileGDB_management(in_folder, "scratch")

featureclasses = arcpy.ListFeatureClasses()
    
print(featureclasses)

# Iterate through all datasets, repair and convert to json
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
        
    # Process: Make Feature Layer (Make Feature Layer) (management)
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    
    # Process: Union - fill gaps and find overlaps
    projectUnion = os.path.join(scr_folder, "projectUnion")
    arcpy.analysis.Union(fLayer, projectUnion, "ALL", None, "NO_GAPS")

    # Process: Multipart To Singlepart - explode into single parts
    projectUnionSingle = os.path.join(scr_folder, "projectUnionSingle")
    arcpy.management.MultipartToSinglepart(projectUnion, projectUnionSingle)
        
    # Process: Select - select overlaps and slithers smaller than 0.00001 degrees (this may need to be checked)
    unionSelect = os.path.join(scr_folder, "unionSelect")
    arcpy.analysis.Select(projectUnionSingle, unionSelect, "Shape_Area <= 0.000006")
    
    # Process: Dissolve - dissolve slithers and overlaps to single features
    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(unionSelect, dissolveSlither, None, None,"SINGLE_PART")

    # Process: Erase overlaps
    removeSlither = os.path.join(scr_folder, "removeSlither")
    arcpy.analysis.Erase(fLayer, dissolveSlither, removeSlither)

    # Process: Append small slithers and gap fills to be merged into coincident polygons with largest boundary
    appendSlither = arcpy.management.Append(dissolveSlither, removeSlither, "NO_TEST")
     
    # Process: Eliminate slither by merge into coincident polygons with largest boundary
    eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
    selectSlither = arcpy.management.SelectLayerByAttribute(appendSlither, "NEW_SELECTION", "OBJECTID IS NULL")
    arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
    print('repairing...')
    
    # Process: Repair Geometry (non-simple geometry)
    geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]
    
    # Process: Features To JSON
    arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
    
    print(outjson, ' json complete')
    
arcpy.Delete_management(scr_folder)
arcpy.AddMessage("All done!")
print('done')